# Representação de Trajetória
O objetivo deste notebook é apresentar os resultados obtidos de maneira visual.

In [1]:
import data_processing
import re

data = data_processing.load_student_data()
disciplinas = data_processing.load_disciplinas()

CSD20
['ICSD20', 'IF61B']
CSF13
['ICSF13', 'IF61C']
FI71S
['FIS7F1', 'FI71M', 'FI71Z']
MA71A
['MAT7C1', 'MA61A', 'MA71Z']
MA71B
['MAT7AL', 'MAT7GA', 'MA61B', 'MA71I', 'MA71Y', 'MA72I']
CSE20
['ICSE20', 'IF62C']
CSF20
['ICSF20', 'IF62E']
EEX11
['ELEX10', 'ELE11']
FI72N
['FIS7E1', 'FI61A', 'FI62A', 'FI71A', 'FI71Y', 'FI72A', 'FI72B']
FI72S
['FIS7F2', 'FI72M', 'FI72Z']
MA72A
['MA62A', 'MA72H', 'MA75D']
QB70C
['QBI7QE', 'QBI7QT', 'QB64A', 'QB70D', 'QB70I']
CSF30
['ICSF30', 'IF63C']
EEB21
['ELEB20', 'ELP21', 'EL63A', 'EL72F']
EEX21
['ELEX20', 'ELE41', 'ELX91']
FI73S
['FIS7F3', 'FI73M']
MA70G
['FI74K', 'MAT7ED', 'MA63B', 'MA70B', 'MA70Z']
MA70H
['EST70A', 'MA65A', 'MA70F']
CSD21
['ICSD21', 'IF63E']
EEB22
['ELEB21', 'ELP32', 'EL64F', 'EL73F']
EEB31
['ELEB30', 'ELF41', 'EL65A', 'EL75F']
EEF11
['ELB13', 'ELEF10', 'EL62A']
EEQ31
['ELB66', 'ELEQ30', 'EL65D']
FI74S
['FIS7F4', 'FI74M']
CSA30
['ICSA30', 'IF64C']
CSB30
['ICSB30', 'IF65E']
CSG20
['ICSG20', 'IF65D']
CSW30
['ELEW30', 'ELF61', 'EL66C']
E

In [2]:
for item in disciplinas:
    if re.search('P[1-9]', item):
        print(item)

In [3]:
sugestoes = []
with open('sugestoes.txt') as file:
    for line in file:
        nome = line.strip().split('\n')[0]
        if nome in disciplinas:
            disc = disciplinas[nome]
            sugestoes.append(disc)
            if disc.trilha == disc.trilha == 'obrigatoria':
                print(f'{disc.nome} não está em nenhuma trilha?')
        else:
            print(f'{nome} não está em disciplinas')

In [4]:
from typing import List

def dar_sugestoes(CHS_desejada : int, sugestoes : List, id_aluno : int, dados = None):
    # Carga horária desejada
    CH_desejada = CHS_desejada * 15
    if len(dados) == 0:
        dados_aluno = (data['todos'][data['todos']['ID_ANONIMO'] == id_aluno])
    else:
        dados_aluno = (dados[dados['ID_ANONIMO'] == id_aluno])

    dados_aluno = dados_aluno[dados_aluno['NOTA'] >= 6]
    disciplinas_cursadas = dados_aluno['CODIGO'].unique()
    it = 0

    for disc in disciplinas:
        i = 0
        for item in disciplinas_cursadas:
            if item in disciplinas[disc].equivalentes:
                disciplinas_cursadas[i] = disc
                break
            i += 1

    sugest_aluno = sugestoes.copy()

    trilha_1 = None
    trilha_2 = None

    obrigatorias_por_periodo = []
    ch_trilha1 = 0
    ch_trilha2 = 0
    ch_isoladas = 0

    for i in range(0,12):
        obrigatorias_por_periodo.append([])

    for disc in disciplinas:
        try:
            if disciplinas[disc].trilha == 'obrigatoria' and disc not in disciplinas_cursadas and disc not in ["CSX53"]:
                print(f'obrigatoria nao cursada: {disc}')
                obrigatorias_por_periodo[disciplinas[disc].periodo - 1].append(disc)
                #print(disciplinas[disc].codigo)
            elif disc in disciplinas_cursadas and disciplinas[disc].trilha != 'obrigatoria':
                if ((trilha_1 == None and disciplinas[disc].trilha != 'Optativas Isoladas') or disciplinas[disc].trilha == trilha_1):
                    trilha_1 = disciplinas[disc].trilha
                    ch_trilha1 += disciplinas[disc].ch
                    #print(f'trilha 1 : {trilha_1}')
                elif ((trilha_2 == None and disciplinas[disc].trilha != trilha_1 and disciplinas[disc].trilha != 'Optativas Isoladas') or disciplinas[disc].trilha == trilha_2): 
                    trilha_2 = disciplinas[disc].trilha
                    #print(f'trilha 2 : {trilha_2}')
                    ch_trilha2 += disciplinas[disc].ch
                else:
                    #print(f'{disciplinas[disc].codigo}, {disciplinas[disc].trilha}')
                    ch_isoladas += disciplinas[disc].ch
        except KeyError:
            print(f'Key error: {disc}')
    # Inicio algoritmo guloso
    formado = False
    periodo_atual = 0
    semestre_atual = 0
    ch_atual = 0
    disciplinas_semestre = []
    disciplinas_semestre.append([])

    # Priorizamos sempre colocar as obrigatórias antes
    for periodo_atual in range(0,11):
        print(f'periodo atual {periodo_atual}')
        for disc in obrigatorias_por_periodo[periodo_atual]:
            print(f'obrigatoria nao cursada - processando: {disc}')
            disc = disciplinas[disc]
            #print(disc.codigo)
            #print(disc.nome)
            ch_atual += disc.ch
            disciplinas_semestre[semestre_atual].append(disc.codigo)
            obrigatorias_por_periodo[periodo_atual].remove(disc.codigo)
            if ch_atual > CH_desejada:
                disciplinas_semestre.append([])
                semestre_atual += 1
                ch_atual = 0

    for disc in sugest_aluno:
        if ch_trilha1 >= 90 and ch_trilha2 >= 90 and ch_isoladas >= 90:
            #print('ch opts concluida')
            break
        if (trilha_1 == None or disciplinas[disc].trilha == trilha_1)  and ch_trilha1 < 90 and disciplinas[disc].trilha != 'Optativas Isoladas':
            #print(f'trilha 1 : {trilha_1}')
            trilha_1 = disciplinas[disc].trilha
            disciplinas_semestre[semestre_atual].append(disc)
            sugest_aluno.remove(disc)
            ch_trilha1 += disciplinas[disc].ch
            ch_atual += disciplinas[disc].ch
        elif (trilha_2 == None or disciplinas[disc].trilha == trilha_2) and ch_trilha2 < 90 and disciplinas[disc].trilha != 'Optativas Isoladas':
            #print(f'trilha 2 : {trilha_2}')
            trilha_2 = disciplinas[disc].trilha
            disciplinas_semestre[semestre_atual].append(disc)
            sugest_aluno.remove(disc)
            ch_trilha2 += disciplinas[disc].ch
            ch_atual += disciplinas[disc].ch
        elif ch_isoladas < 90 and disciplinas[disc].trilha != trilha_1 and disciplinas[disc].trilha != trilha_2:
            disciplinas_semestre[semestre_atual].append(disc)
            sugest_aluno.remove(disc)
            ch_isoladas += disciplinas[disc].ch
            ch_atual += disciplinas[disc].ch

        if ch_atual > CH_desejada:
            disciplinas_semestre.append([])
            ch_atual = 0
            semestre_atual += 1

    return disciplinas_semestre



In [5]:
disciplinas['EEC31'].periodo

8

In [6]:
matriz = data_processing.get_matriz()

In [7]:
with open('legenda.csv', 'w', encoding='UTF-8') as f:
    for item in matriz.itertuples():
        f.write(f'{item[3]},{item[4]}\n\n')

## Testes

In [8]:
from data_processing import get_all_data_from_student, trim_user_data, get_course_list, user_based_suggestions, select_random_student, create_cache_user_arrays

In [9]:
user_id = select_random_student(data['formados'], 20, 2015)
user_data = get_all_data_from_student(user_id, data['formados'])

In [10]:
cache = create_cache_user_arrays(data["todos"], centralize_cosine = True)

### Calculamos precision e recall para os estudantes formados, removendo as últimas 12 disciplinas realizadas (aprox. 3 semestres)

In [11]:
import sys
sys.path.insert(0,'../utils')
from recommendation import recommend_dis, get_disciplinas_variables
_,_,df_844_optativas = get_disciplinas_variables()

../excel/dados_historicos_alunos_de_EngComputacaoCuritiba formados.csv
../excel/dados_historicos_alunos_de_EngComputacaoCuritiba regulares.csv


In [12]:
import pandas as pd
def trim_ate_optativa(num_optativas : int, dados_estudante : pd.DataFrame):
    achou = 0
    ind = 0

    df = dados_estudante.sort_values(['ANO','PERIODO.1'],axis=0)
    # Reset the index to add a new column with the default integer index
    df['RowNumber'] = range(1, len(df) + 1)
    df.set_index('RowNumber')

    for item in df.itertuples():
        if item[5] in list(df_844_optativas["CODIGO"]):
            print(item[5])
            print(disciplinas[item[5]].trilha)
            achou += 1
        if achou >= num_optativas:
            ind = item[0]
            ano = item[6]
            periodo = item[7]
            #print(f'{ano}  {periodo}')
            #print(ind)
            break

    if ind > 0:
        df = df[df['ANO'] <= ano]#int(ano.iloc[0])]
        if periodo == 1:
            df = df[~((df['ANO'] == ano) & (df['PERIODO.1'] == 2))]
        return df
    else:
        return df

In [13]:
from tqdm import tqdm
import concurrent.futures
from threading import Lock

class Log:
    path = ''
    mutex_lock = Lock()

    def clear(self):
        self.mutex_lock.acquire()
        with open(self.path, 'w') as f:
            f.write('')
        self.mutex_lock.release()


    def write(self, log):
        self.mutex_lock.acquire()
        with open(self.path, 'a') as f:
            f.write(log)
            f.write('\n')
        self.mutex_lock.release()

        

    def __init__(self, path):
        self.path = path

def teste_recomendacao(num_disciplinas_trim : int, cache):
    class Metrica:
        total = 0
        media = 0
        minimo = 0
        maximo = 0
        num_itens = 0

        def add_item(self, item : int):
            self.num_itens += 1
            if self.minimo > item:
                self.minimo = item
            if self.maximo < item:
                self.maximo = item
            self.total += item
            self.media = self.total / self.num_itens
        
        def print(self):
            print(f'Media: {self.media}\nMinimo: {self.minimo}\nMaximo: {self.maximo}\n')
        
    class Results:
        sugestoes = []
        sugestoes_alg = []
        estudante = 0
        precision = 0
        recall = 0
        true_positives = 0
        disciplinas_com_erro = []
        precision_ob = 0
        recall_ob = 0
        precision_op = 0
        recall_op = 0

        def __init__(self, sugestoes,sugestoes_alg,estudante,precision,recall,true_positives,disciplinas_com_erro,p_ob,r_ob,p_op,r_op):
            self.sugestoes = sugestoes
            self.sugestoes_alg = sugestoes_alg
            self.estudante = estudante
            self.precision = precision
            self.recall = recall
            self.true_positives = true_positives
            self.disciplinas_com_erro = disciplinas_com_erro
            self.precision_ob = p_ob
            self.recall_ob = r_ob
            self.precision_op = p_op
            self.recall_op = r_op
    

    log = Log('output/teste_rec.txt')
    log.clear()

    res_csv = Log('output/res.csv')
    res_csv.clear()

    def turn_to_date(x : str):
        try:
            x = int(x.split('/')[2][:2])
        except:
            #print(x)
            x = 0
        return x
            

    #data['formados']['INGRESSO'].map(turn_to_date)
    dados = data['formados'][data['formados']['INGRESSO'].map(turn_to_date) > 14]
    lista_estudantes = dados['ID_ANONIMO'].unique()
    #print(lista_estudantes)

    def thread_estudante(num_disciplinas_trim, estudante):
        #log.write(f'{i} started')
        #pbar.update(1)
        dados_estudante = get_all_data_from_student(int(estudante), data['formados'])
        if len(dados_estudante) == 0:
            print(dados_estudante)
            return 0
        dados_trimmed = trim_ate_optativa(dados_estudante, num_disciplinas_trim)
        if len(dados_trimmed.index) == 0:
            print(dados_trimmed)
            return 0
        # Sugerimos disciplinas com base nos dados tirando as ultimas num_disciplinas_trim disciplinas
        sugestoes = [key for key in dict(sorted(user_based_suggestions(estudante, data_rec=data['formados'], data_user=dados_trimmed,cache=cache, centralize_cosine=True, reduce_popularity_weight=False),key=lambda x:x[1], reverse=True))]
        #print(estudante)
        #print(sugestoes)
        if len(sugestoes) == 0:
            print(sugestoes)
            return 0
        
        discs_trimmed = list(trim_user_data(dados_estudante, num_disciplinas_trim)['CODIGO'].unique())
        sugestoes_alg = dar_sugestoes(20, sugestoes, estudante, dados_trimmed)
        disciplinas_alg = []
        for periodo in sugestoes_alg:
            for disc in periodo:
                disciplinas_alg.append(disc)
        if len(disciplinas_alg) == 0:
            print (estudante)
            return 0

        # Montar lista de equivalentes
        equivalentes_estudante = []
        disciplinas_com_erro = []
        lista_disc = [disc for disc in list(dados_estudante['CODIGO'].unique()) if disc not in discs_trimmed]
        for disc in lista_disc:
            try:
                for eq in disciplinas[disc].equivalentes:
                    equivalentes_estudante.append(eq)
            except KeyError:
                disciplinas_com_erro.append(disc)

        # Calcular precisao e recall
        true_positives = 0
        truep_ob = 0
        truep_op = 0
        len_alg_ob = 0
        len_list_ob = 0
        for disc in disciplinas_alg:
            if disc in lista_disc or disc in equivalentes_estudante:
                true_positives += 1
                if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                    truep_ob += 1
                else:
                    truep_op += 1
            if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                len_alg_ob += 1

        for disc in lista_disc:
            if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                len_list_ob += 1
        
        result_precision=(true_positives/len(disciplinas_alg))
        result_recall=(true_positives/len(lista_disc))

        len_alg_op = len(disciplinas_alg) - len_alg_ob
        if len_alg_op == 0:
            len_alg_op = float('inf')
        if len_alg_ob == 0:
            len_alg_ob = float('inf')
        len_list_op = len(lista_disc) - len_alg_ob
        if len_list_op == 0:
            len_list_op = float('inf')
        if len_list_ob == 0:
            len_list_ob = float('inf')

        p_ob = truep_ob/len_alg_ob
        r_ob = truep_ob/len_list_ob

        p_op = truep_op/len_alg_op
        r_op = truep_op/len_list_op

        return Results(sugestoes,sugestoes_alg,estudante,result_recall,result_precision,true_positives, disciplinas_com_erro,p_ob,r_ob,p_op,r_op)
        
    
    pool = concurrent.futures.ThreadPoolExecutor(max_workers = 22)
    print(lista_estudantes)
    futures = [pool.submit(thread_estudante, num_disciplinas_trim,estudante) for estudante in lista_estudantes]

    print(f'waiting for {len(lista_estudantes)} threads')


    precision = Metrica()
    recall = Metrica()
    precision_ob = Metrica()
    recall_ob = Metrica()
    precision_op = Metrica()
    recall_op = Metrica()

    disciplinas_com_erro = []
    res_csv.write('ID_ANONIMO,PRECISION,RECALL,PRECISION_OB,RECALL_OB,PRECISION_OP,RECALL_OP')
    for i in tqdm(range(0,len(lista_estudantes))):
            item = futures[i]
            #item = lista_estudantes[i]
            result : Results = item.result() #thread_estudante(num_disciplinas_trim, item)
            if result != 0:
                precision.add_item(result.precision)
                recall.add_item(result.recall)
                precision_ob.add_item(result.precision_ob)
                recall_ob.add_item(result.recall_ob)
                precision_op.add_item(result.precision_op)
                recall_op.add_item(result.recall_op)
                log.write(f'\n{result.estudante} done\nTrue Positives: {result.true_positives}\nSugestoes: {result.sugestoes}\nSugestoes com pos proc: {result.sugestoes_alg}\nRecall: {result.recall}\nPrecision: {result.precision}\nRecall Obrigatorias: {result.recall_ob}\nPrecision Obrigatorias: {result.precision_ob}\nRecall Optativas: {result.recall_op}\nPrecision Optativas: {result.precision_op}\n')
                for disc in result.disciplinas_com_erro:
                    if disc not in disciplinas_com_erro:
                        disciplinas_com_erro.append(disc)
                res_csv.write(f'{int(result.estudante)},{result.precision},{result.recall},{result.precision_ob},{result.recall_ob},{result.precision_op},{result.recall_op}')
    #pool.shutdown(wait=True)
    return recall, precision, recall_ob, precision_ob, recall_op, precision_op
    




In [14]:
def turn_to_date(x : str):
    try:
        x = int(x.split('/')[2][:2])
    except:
        #print(x)
        x = 0
    return x

dados = data['formados'][data['formados']['INGRESSO'].map(turn_to_date) > 14]
lista_estudantes = dados['ID_ANONIMO'].unique()

for student in lista_estudantes:
    dados_estudante = dados[dados['ID_ANONIMO'] == student]
    dados_estudante.to_csv(f'input/dados_estudante/full/{int(student)}.csv')
    dados_estudante = trim_user_data(dados_estudante, 6)
    dados_estudante.to_csv(f'input/dados_estudante/trimmed/{int(student)}_trimmed_{6}.csv')


In [15]:
sugest = recommend_dis('input/dados_estudante/trimmed/150878_trimmed_2.csv','CliquesMaximais')
print(sugest)

CSV path: input/dados_estudante/trimmed/150878_trimmed_2.csv
Alg selecionado: CliquesMaximais
DiGraph with 55 nodes and 433 edges
[['CSH30', 1.3966480446927374], ['CSH42', 1.0614525139664805], ['CSM41', 0.8379888268156424], ['CSM43', 0.8379888268156424], ['DI84D', 0.5586592178770949], ['CSM44', 0.5027932960893855], ['ES70B', 0.446927374301676], ['CSM40', 0.3910614525139665], ['CSR41', 0.33519553072625696], ['ED70T', 0.27932960893854747], ['ED70U', 0.27932960893854747], ['CSH43', 0.27932960893854747], ['CSR44', 0.223463687150838], ['CSI53', 0.16759776536312848], ['CSB41', 0.16759776536312848], ['EL64B', 0.16759776536312848], ['EL6AE', 0.16759776536312848], ['CSH44', 0.16759776536312848], ['CSB51', 0.111731843575419], ['CSB53', 0.111731843575419], ['CSR47', 0.111731843575419], ['FCH7HB', 0.111731843575419], ['CSH41', 0.111731843575419], ['CSI58', 0.0558659217877095], ['CSI31', 0.0558659217877095], ['CSB54', 0.0558659217877095], ['CSI56', 0.0558659217877095], ['CSR43', 0.0558659217877095]

In [16]:
lista_sugestoes = [item[0] for item in sugest]
print(lista_sugestoes)

['CSH30', 'CSH42', 'CSM41', 'CSM43', 'DI84D', 'CSM44', 'ES70B', 'CSM40', 'CSR41', 'ED70T', 'ED70U', 'CSH43', 'CSR44', 'CSI53', 'CSB41', 'EL64B', 'EL6AE', 'CSH44', 'CSB51', 'CSB53', 'CSR47', 'FCH7HB', 'CSH41', 'CSI58', 'CSI31', 'CSB54', 'CSI56', 'CSR43', 'CSR42', 'CSG42', 'IF6BV', 'MA70C', 'CSD41']


In [17]:
print(df_844_optativas["CODIGO"])
print(list(df_844_optativas["CODIGO"]))

20721    CSI58
20726    EEY51
20727    CSB51
20738    EEL51
20752    CSB41
         ...  
49660    FI70D
49662    GE70F
49663    MA70C
49675    QB70F
49689    CSE47
Name: CODIGO, Length: 316, dtype: object
['CSI58', 'EEY51', 'CSB51', 'EEL51', 'CSB41', 'CSI53', 'CSB53', 'CSV30', 'CSV40', 'CSR41', 'CSR48', 'CSM40', 'ES70B', 'ES70J', 'ES70N', 'CSH30', 'CSH44', 'CSM45', 'CSM41', 'CSB54', 'CSI31', 'ES70J', 'ES70N', 'CSI53', 'CSB41', 'ED70T', 'CSM41', 'ES70B', 'ES70J', 'ES70N', 'CSH30', 'CSM43', 'CSH42', 'FI70D', 'FI70A', 'ED70U', 'CSH42', 'CSH30', 'CSB41', 'IF6BV', 'GE70F', 'CSM40', 'CSM43', 'CSH44', 'CSH30', 'MA70C', 'FI70A', 'EL64B', 'CSB53', 'CSB51', 'CSB51', 'CSI53', 'MA70C', 'FI70D', 'CSR42', 'EEY44', 'EEY41', 'EEY41', 'CSR41', 'EEY44', 'ED70T', 'CSV30', 'CSB53', 'CSM44', 'CSR41', 'CSB51', 'CSM44', 'CSB41', 'CSV30', 'ED70T', 'CSB54', 'CSV40', 'ED70U', 'ES70N', 'CSM44', 'CSR41', 'CSI53', 'ED70T', 'CSM44', 'CSM40', 'CSR44', 'CSV30', 'CSB54', 'CSV40', 'CSV30', 'ED70T', 'CSB41', 'CSA41', '

In [18]:
#trim_ate_optativa(1, get_all_data_from_student(182913, dados))
teste = get_all_data_from_student(150878,dados)
teste = trim_ate_optativa(9,teste)
print(teste)

ES70J
Optativas Isoladas
ES70N
Optativas Isoladas
ES70B
Optativas Isoladas
CSM41
Desenvolvimento Baseado em Plataformas Programacao
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSH30
IHC
CSH44
IHC
       ID_ANONIMO     CR  PERIODO  INGRESSO CODIGO   ANO  PERIODO.1  NOTA   CH SITUACAOALUNO  RowNumber
21856    150878.0  0.733       10  02/03/17  FI71M  2017          1   6.0   60       Formado          1
21860    150878.0  0.733       10  02/03/17  MA71A  2017          1   7.5   90       Formado          2
21867    150878.0  0.733       10  02/03/17  MA71B  2017          1   7.5   90       Formado          3
21878    150878.0  0.733       10  02/03/17  CSF13  2017          1   6.0   90       Formado          4
21903    150878.0  0.733       10  02/03/17  CSD20  2017          1   7.5   45       Formado          5
...           ...    ...      ...       ...    ...   ...        ...   ...  ...           ...        ...
21928    150878.0  0.733       10  02/03/17  CL74A  2022       

In [19]:
class Metrica:
        total = 0
        media = 0
        minimo = 0
        maximo = 0
        num_itens = 0

        def add_item(self, item : int):
            self.num_itens += 1
            if self.minimo > item:
                self.minimo = item
            if self.maximo < item:
                self.maximo = item
            self.total += item
            self.media = self.total / self.num_itens
        
        def print(self):
            print(f'Media: {self.media}\nMinimo: {self.minimo}\nMaximo: {self.maximo}\n')
        
class Results:
    sugestoes = []
    sugestoes_alg = []
    estudante = 0
    precision = 0
    recall = 0
    true_positives = 0
    disciplinas_com_erro = []
    precision_ob = 0
    recall_ob = 0
    precision_op = 0
    recall_op = 0

    def __init__(self, sugestoes,sugestoes_alg,estudante,precision,recall,true_positives,disciplinas_com_erro,p_ob,r_ob,p_op,r_op):
        self.sugestoes = sugestoes
        self.sugestoes_alg = sugestoes_alg
        self.estudante = estudante
        self.precision = precision
        self.recall = recall
        self.true_positives = true_positives
        self.disciplinas_com_erro = disciplinas_com_erro
        self.precision_ob = p_ob
        self.recall_ob = r_ob
        self.precision_op = p_op
        self.recall_op = r_op
def thread_estudante(num_disciplinas_trim, estudante, alg):
        #log.write(f'{i} started')
        #pbar.update(1)
        dados_estudante = get_all_data_from_student(int(estudante), data['formados'])
        if len(dados_estudante) == 0:
            print(dados_estudante)
            return 0
        dados_trimmed = trim_ate_optativa(num_disciplinas_trim,dados_estudante)
        if len(dados_trimmed.index) == 0:
            print(dados_trimmed)
            return 0
        # Sugerimos disciplinas com base nos dados tirando as ultimas num_disciplinas_trim disciplinas
        if alg != 'cosseno':
            sugestoes = [item[0] if len(item) > 1 else item for item in recommend_dis(f'input/dados_estudante/trimmed/{int(estudante)}_trimmed_{num_disciplinas_trim}.csv', alg) ]
        else:
            sugestoes = [key for key in dict(sorted(user_based_suggestions(estudante, data_rec=data['formados'], data_user=dados_trimmed,cache=cache, centralize_cosine=True, reduce_popularity_weight=False),key=lambda x:x[1], reverse=True))]
        #print(estudante)
        #print(sugestoes)
        if len(sugestoes) == 0:
            print(sugestoes)
            return 0
        
        discs_trimmed = list(dados_trimmed['CODIGO'].unique())
        sugestoes_alg = dar_sugestoes(20, sugestoes, estudante, dados_trimmed)
        disciplinas_alg = []
        for periodo in sugestoes_alg:
            for disc in periodo:
                disciplinas_alg.append(disc)
        if len(disciplinas_alg) == 0:
            print (estudante)
            return 0

        # Montar lista de equivalentes
        equivalentes_estudante = []
        disciplinas_com_erro = []
        lista_disc = [disc for disc in list(dados_estudante['CODIGO'].unique()) if disc not in discs_trimmed]
        for disc in lista_disc:
            try:
                for eq in disciplinas[disc].equivalentes:
                    equivalentes_estudante.append(eq)
            except KeyError:
                disciplinas_com_erro.append(disc)

        # Calcular precisao e recall
        true_positives = 0
        truep_ob = 0
        truep_op = 0
        len_alg_ob = 0
        len_list_ob = 0
        for disc in disciplinas_alg:
            if disc in lista_disc or disc in equivalentes_estudante:
                true_positives += 1
                if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                    truep_ob += 1
                else:
                    truep_op += 1
            if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                len_alg_ob += 1

        for disc in lista_disc:
            if disc in disciplinas.keys() and disciplinas[disc].trilha == 'obrigatoria':
                len_list_ob += 1
        
        if len(disciplinas_alg) == 0:
            len_disc_alg = float('inf')
        else:
            len_disc_alg = len(disciplinas_alg)
        result_precision=(true_positives/len_disc_alg)
        if len(lista_disc) == 0:
            len_lista_disc = float('inf')
        else:
            len_lista_disc = len(lista_disc)
        result_recall=(true_positives/len_lista_disc)

        len_alg_op = len(disciplinas_alg) - len_alg_ob
        if len_alg_op == 0:
            len_alg_op = float('inf')
        if len_alg_ob == 0:
            len_alg_ob = float('inf')
        len_list_op = len(lista_disc) - len_list_ob
        if len_list_op == 0:
            len_list_op = float('inf')
        if len_list_ob == 0:
            len_list_ob = float('inf')

        p_ob = truep_ob/len_alg_ob
        r_ob = truep_ob/len_list_ob

        p_op = truep_op/len_alg_op
        r_op = truep_op/len_list_op

        return Results(sugestoes,sugestoes_alg,estudante,result_recall,result_precision,true_positives, disciplinas_com_erro,p_ob,r_ob,p_op,r_op)

In [20]:
import sys,os
def teste_recomendacao_grafos(num_disciplinas_trim : int, alg : str):
    log = Log('output/teste_rec.txt')
    log.clear()

    res_csv = Log('output/res.csv')
    res_csv.clear()

    def turn_to_date(x : str):
        try:
            x = int(x.split('/')[2][:2])
        except:
            #print(x)
            x = 0
        return x
            
    #data['formados']['INGRESSO'].map(turn_to_date)
    dados = data['formados'][data['formados']['INGRESSO'].map(turn_to_date) > 14]
    lista_estudantes = dados['ID_ANONIMO'].unique()
    for student in lista_estudantes:
        dados_estudante = dados[dados['ID_ANONIMO'] == student]
        dados_estudante.to_csv(f'input/dados_estudante/full/{int(student)}.csv')
        dados_estudante = trim_ate_optativa(num_disciplinas_trim, dados_estudante)
        dados_estudante.to_csv(f'input/dados_estudante/trimmed/{int(student)}_trimmed_{num_disciplinas_trim}.csv')
        #print(lista_estudantes)

    #pool = concurrent.futures.ThreadPoolExecutor(max_workers = 22)
    print(lista_estudantes)
    #futures = [pool.submit(thread_estudante, num_disciplinas_trim,estudante) for estudante in lista_estudantes]

    print(f'waiting for {len(lista_estudantes)} threads')

    precision = Metrica()
    recall = Metrica()
    precision_ob = Metrica()
    recall_ob = Metrica()
    precision_op = Metrica()
    recall_op = Metrica()

    disciplinas_com_erro = []
    res_csv.write('ID_ANONIMO,PRECISION,RECALL,PRECISION_OB,RECALL_OB,PRECISION_OP,RECALL_OP')
    for i in tqdm(range(0,len(lista_estudantes))):
            #item = futures[i]
            item = lista_estudantes[i]
            #result : Results = item.result() 
            #sys.stdout = open(os.devnull, 'w')
            result : Results = thread_estudante(num_disciplinas_trim, item, alg)
            #sys.stdout = sys.__stdout__
            if result != 0:
                precision.add_item(result.precision)
                recall.add_item(result.recall)
                precision_ob.add_item(result.precision_ob)
                recall_ob.add_item(result.recall_ob)
                precision_op.add_item(result.precision_op)
                recall_op.add_item(result.recall_op)
                log.write(f'\n{result.estudante} done\nTrue Positives: {result.true_positives}\nSugestoes: {result.sugestoes}\nSugestoes com pos proc: {result.sugestoes_alg}\nRecall: {result.recall}\nPrecision: {result.precision}\nRecall Obrigatorias: {result.recall_ob}\nPrecision Obrigatorias: {result.precision_ob}\nRecall Optativas: {result.recall_op}\nPrecision Optativas: {result.precision_op}\n')
                for disc in result.disciplinas_com_erro:
                    if disc not in disciplinas_com_erro:
                        disciplinas_com_erro.append(disc)
                res_csv.write(f'{int(result.estudante)},{result.precision},{result.recall},{result.precision_ob},{result.recall_ob},{result.precision_op},{result.recall_op}')
    #pool.shutdown(wait=True)
    print('Recall: \n')
    recall.print()
    print('Precision: \n')
    precision.print()
    print('Recall Obrigatorias: \n')
    recall_ob.print()
    print('Precision Obrigatorias: \n')
    precision_ob.print()
    print('Recall Optativas: \n')
    recall_op.print()
    print('Precision Optativas: \n')
    precision_op.print()



In [21]:
estudante = 172045
num_disciplinas_trim = 2
alg = 'cosseno'

dados_estudante = get_all_data_from_student(int(estudante), data['formados'])
if len(dados_estudante) == 0:
    print(f'len dados estudante: {dados_estudante}')
    
dados_trimmed = trim_ate_optativa(num_disciplinas_trim,dados_estudante)
if len(dados_trimmed.index) == 0:
    print(f'len dados trimmed: {dados_trimmed}')

# Sugerimos disciplinas com base nos dados tirando as ultimas num_disciplinas_trim disciplinas
if alg != 'cosseno':
    sugestoes = [item[0] for item in recommend_dis(f'input/dados_estudante/trimmed/{int(estudante)}_trimmed_{num_disciplinas_trim}.csv', alg)]
else:
    sugestoes = [key for key in dict(sorted(user_based_suggestions(estudante, data_rec=data['formados'], data_user=dados_trimmed,cache=cache, centralize_cosine=True, reduce_popularity_weight=False),key=lambda x:x[1], reverse=True))]
#print(estudante)
#print(sugestoes)
if len(sugestoes) == 0:
    print(f'sugestoes: {sugestoes}')

discs_trimmed = list(dados_trimmed['CODIGO'].unique())
sugestoes_alg = dar_sugestoes(20, sugestoes, estudante, dados_trimmed)
disciplinas_alg = []
for periodo in sugestoes_alg:
    for disc in periodo:
        disciplinas_alg.append(disc)
if len(disciplinas_alg) == 0:
    print (f' estudante: {estudante}')

# Montar lista de equivalentes
equivalentes_estudante = []
disciplinas_com_erro = []
lista_disc = [disc for disc in list(dados_estudante['CODIGO'].unique()) if disc not in discs_trimmed]
for disc in lista_disc:
    try:
        for eq in disciplinas[disc].equivalentes:
            equivalentes_estudante.append(eq)
    except KeyError:
        disciplinas_com_erro.append(disc)

ES70N
Optativas Isoladas
CSH30
IHC
obrigatoria nao cursada: CSE30
obrigatoria nao cursada: CSO30
obrigatoria nao cursada: CSW40
obrigatoria nao cursada: CSX54
obrigatoria nao cursada: EEE32
obrigatoria nao cursada: CSI30
obrigatoria nao cursada: CSR30
obrigatoria nao cursada: CSW41
obrigatoria nao cursada: CSW42
obrigatoria nao cursada: EEC21
obrigatoria nao cursada: EEQ32
obrigatoria nao cursada: CSS30
obrigatoria nao cursada: EEC31
obrigatoria nao cursada: EEX23
obrigatoria nao cursada: CSX42
obrigatoria nao cursada: GE70H
obrigatoria nao cursada: QB70E
obrigatoria nao cursada: CSX43
obrigatoria nao cursada: EEF31
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
obrigatoria nao cursada - processando: CSE30
obrigatoria nao cursada - processando: CSW40
obrigatoria nao cursada - processando: EEE32
periodo atual 6
obrigatoria nao cursada - processando: CSI30
obrigatoria nao cursada - processando: CSW41
obrigatoria nao cursada - processando: 

In [22]:
sugestoes_alg

[['CSE30', 'CSW40', 'EEE32', 'CSI30', 'CSW41', 'EEC21'],
 ['CSS30', 'EEX23', 'CSX42', 'QB70E', 'CSX43', 'CSH42']]

In [23]:
result = thread_estudante(2, 172045,'cosseno')
print(f'{int(result.estudante)},{result.precision},{result.recall},{result.precision_ob},{result.recall_ob},{result.precision_op},{result.recall_op}')

ES70N
Optativas Isoladas
CSH30
IHC
obrigatoria nao cursada: CSE30
obrigatoria nao cursada: CSO30
obrigatoria nao cursada: CSW40
obrigatoria nao cursada: CSX54
obrigatoria nao cursada: EEE32
obrigatoria nao cursada: CSI30
obrigatoria nao cursada: CSR30
obrigatoria nao cursada: CSW41
obrigatoria nao cursada: CSW42
obrigatoria nao cursada: EEC21
obrigatoria nao cursada: EEQ32
obrigatoria nao cursada: CSS30
obrigatoria nao cursada: EEC31
obrigatoria nao cursada: EEX23
obrigatoria nao cursada: CSX42
obrigatoria nao cursada: GE70H
obrigatoria nao cursada: QB70E
obrigatoria nao cursada: CSX43
obrigatoria nao cursada: EEF31
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
obrigatoria nao cursada - processando: CSE30
obrigatoria nao cursada - processando: CSW40
obrigatoria nao cursada - processando: EEE32
periodo atual 6
obrigatoria nao cursada - processando: CSI30
obrigatoria nao cursada - processando: CSW41
obrigatoria nao cursada - processando: 

In [24]:
import pandas as pd

In [25]:
teste = pd.read_csv('output/res.csv')
print(teste.describe())
print(teste.drop(teste[teste['RECALL_OP'] == 0].index).describe())

          ID_ANONIMO  PRECISION     RECALL  PRECISION_OB  RECALL_OB  PRECISION_OP  RECALL_OP
count      60.000000  60.000000  60.000000     60.000000  60.000000     60.000000  60.000000
mean   168109.383333   0.190678   0.212222      0.316667   0.226389      0.016667   0.007937
std     24208.484830   0.300697   0.320896      0.450674   0.331950      0.090510   0.046530
min     90729.000000   0.000000   0.000000      0.000000   0.000000      0.000000   0.000000
25%    154922.750000   0.000000   0.000000      0.000000   0.000000      0.000000   0.000000
50%    174468.000000   0.000000   0.000000      0.000000   0.000000      0.000000   0.000000
75%    183413.250000   0.400000   0.500000      1.000000   0.500000      0.000000   0.000000
max    202261.000000   1.000000   1.000000      1.000000   1.000000      0.500000   0.333333
          ID_ANONIMO  PRECISION    RECALL  PRECISION_OB  RECALL_OB  PRECISION_OP  RECALL_OP
count       2.000000   2.000000  2.000000           2.0   2.000000     

In [26]:
for i in range(1,8):
    teste_recomendacao_grafos(i,'Comunidades')
    teste = pd.read_csv('output/res.csv')
    teste.describe().to_csv(f'output/desc_comunidades_{i}.txt')
    teste.drop(teste[teste['RECALL_OP'] == 0].index).describe().to_csv(f'output/desc_notnull_comunidades_{i}.txt')

GE70F
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
CSH30
IHC
CSR44
Redes De Computadores
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
ED70T
Optativas Isoladas
EEY51
Engenharia Biomedica
CSV30
Processamento Grafico
ES70J
Optativas Isoladas
ED70T
Optativas Isoladas
CSM43
Desenvolvimento Baseado em Plataformas Programacao
FI70D
Fisica
CSM40
Desenvolvimento Baseado em Plataformas Programacao
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSH30


  0%|          | 0/68 [00:00<?, ?it/s]

GE70F
Optativas Isoladas
CSV path: input/dados_estudante/trimmed/90729_trimmed_1.csv
Alg selecionado: Comunidades
DiGraph with 57 nodes and 260 edges
(0, 0)
(1, 2)


IndexError: list index out of range

In [ ]:
for i in range(1,8):
    teste_recomendacao_grafos(i,'CliquesMaximais')
    teste = pd.read_csv('output/res.csv')
    teste.describe().to_csv(f'output/desc_cliquesmax_{i}.txt')
    teste.drop(teste[teste['RECALL_OP'] == 0].index).describe().to_csv(f'output/desc_notnull_cliquesmax_{i}.txt')

GE70F
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
CSH30
IHC
CSR44
Redes De Computadores
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
ED70T
Optativas Isoladas
EEY51
Engenharia Biomedica
CSV30
Processamento Grafico
ES70J
Optativas Isoladas
ED70T
Optativas Isoladas
CSM43
Desenvolvimento Baseado em Plataformas Programacao
FI70D
Fisica
CSM40
Desenvolvimento Baseado em Plataformas Programacao
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSH30


  0%|          | 0/68 [00:00<?, ?it/s]

GE70F
Optativas Isoladas
CSV path: input/dados_estudante/trimmed/90729_trimmed_1.csv
Alg selecionado: CliquesMaximais
DiGraph with 57 nodes and 260 edges
obrigatoria nao cursada: FI71S
obrigatoria nao cursada: CSF20
obrigatoria nao cursada: FI72S
obrigatoria nao cursada: QB70C
obrigatoria nao cursada: EEX11
obrigatoria nao cursada: MA73A
obrigatoria nao cursada: FI73S
obrigatoria nao cursada: EEB21
obrigatoria nao cursada: EEX21
obrigatoria nao cursada: EEF11
obrigatoria nao cursada: EEB22
obrigatoria nao cursada: EEQ31
obrigatoria nao cursada: FI74N
obrigatoria nao cursada: FI74S
obrigatoria nao cursada: CSR31
obrigatoria nao cursada: EEE31
obrigatoria nao cursada: EEF21
obrigatoria nao cursada: ES70G
obrigatoria nao cursada: CSO30
obrigatoria nao cursada: CSW40
obrigatoria nao cursada: CSX54
obrigatoria nao cursada: EEE32
obrigatoria nao cursada: EEX22
obrigatoria nao cursada: EEC21
obrigatoria nao cursada: ES70P
obrigatoria nao cursada: CSS30
obrigatoria nao cursada: EEC31
obrigator

  4%|▍         | 3/68 [00:00<00:02, 24.42it/s]

obrigatoria nao cursada: EEX11
obrigatoria nao cursada: EEB22
obrigatoria nao cursada: EEB31
obrigatoria nao cursada: CSB30
obrigatoria nao cursada: CSG20
obrigatoria nao cursada: CSR31
obrigatoria nao cursada: CSW30
obrigatoria nao cursada: EEE31
obrigatoria nao cursada: EEF21
obrigatoria nao cursada: ES70G
obrigatoria nao cursada: CSA31
obrigatoria nao cursada: CSE30
obrigatoria nao cursada: CSO30
obrigatoria nao cursada: CSW40
obrigatoria nao cursada: CSX54
obrigatoria nao cursada: EEE32
obrigatoria nao cursada: CSI30
obrigatoria nao cursada: CSR30
obrigatoria nao cursada: CSW41
obrigatoria nao cursada: CSW42
obrigatoria nao cursada: EEC21
obrigatoria nao cursada: EEQ32
obrigatoria nao cursada: CSS30
obrigatoria nao cursada: EEC31
obrigatoria nao cursada: EEX23
obrigatoria nao cursada: GE70D
obrigatoria nao cursada: CSX42
obrigatoria nao cursada: GE70H
obrigatoria nao cursada: QB70E
obrigatoria nao cursada: CSX43
obrigatoria nao cursada: EEF31
periodo atual 0
periodo atual 1
obrigat

...

obrigatoria nao cursada: CSW41
obrigatoria nao cursada: CSW42
obrigatoria nao cursada: CSX43
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
obrigatoria nao cursada - processando: CSW41
periodo atual 7
periodo atual 8
periodo atual 9
obrigatoria nao cursada - processando: CSX43
periodo atual 10
CSB54
Banco De Dados
CSB41
Banco De Dados
CSV30
Processamento Grafico
ED70T
Optativas Isoladas
CSV40
Processamento Grafico
CSV path: input/dados_estudante/trimmed/190508_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 75 nodes and 541 edges
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
periodo atual 7
periodo atual 8
periodo atual 9
periodo atual 10
CSB53
Banco De Dados
CSB51
Banco De Dados
CSV30
Processamento Grafico
CSA41
Algoritmos E Complexidade
CSA42
Algoritmos E Complexidade
CSA45
Algoritmos E Complexidade
CSV path: input/dados_estudante/trimmed/1906

 97%|█████████▋| 66/68 [00:02<00:00, 26.45it/s]

FI70A
Fisica
CSH30
IHC
CSH42
IHC
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSV path: input/dados_estudante/trimmed/191284_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 72 nodes and 494 edges
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
periodo atual 7
periodo atual 8
periodo atual 9
periodo atual 10
ES70J
Optativas Isoladas
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSM40
Desenvolvimento Baseado em Plataformas Programacao
CSM43
Desenvolvimento Baseado em Plataformas Programacao
CSH30
IHC
CSM41
Desenvolvimento Baseado em Plataformas Programacao
CSH42
IHC
CSV path: input/dados_estudante/trimmed/198372_trimmed_7.csv
Alg selecionado: CliquesMaximais
DiGraph with 74 nodes and 580 edges
obrigatoria nao cursada: CSX43
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
perio

100%|██████████| 68/68 [00:02<00:00, 25.82it/s]

obrigatoria nao cursada: CSX54
obrigatoria nao cursada: CSS30
obrigatoria nao cursada: EEC31
obrigatoria nao cursada: EEX23
obrigatoria nao cursada: CSX42
obrigatoria nao cursada: CSX43
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
obrigatoria nao cursada - processando: CSX54
periodo atual 6
periodo atual 7
obrigatoria nao cursada - processando: CSS30
obrigatoria nao cursada - processando: EEX23
periodo atual 8
obrigatoria nao cursada - processando: CSX42
periodo atual 9
obrigatoria nao cursada - processando: CSX43
periodo atual 10
Recall: 

Media: 0.21222222222222223
Minimo: 0
Maximo: 1.0

Precision: 

Media: 0.1906782106782107
Minimo: 0
Maximo: 1.0

Recall Obrigatorias: 

Media: 0.2263888888888889
Minimo: 0
Maximo: 1.0

Precision Obrigatorias: 

Media: 0.31666666666666665
Minimo: 0
Maximo: 1.0

Recall Optativas: 

Media: 0.007936507936507936
Minimo: 0
Maximo: 0.3333333333333333

Precision Optativas: 

Media: 0.016666666666666666
Minim

In [ ]:
for i in range(1,8):
    teste_recomendacao_grafos(i,'Kcliques')
    teste = pd.read_csv('output/res.csv')
    teste.describe().to_csv(f'output/desc_kcliques_{i}.txt')
    teste.drop(teste[teste['RECALL_OP'] == 0].index).describe().to_csv(f'output/desc_notnull_kcliques_{i}.txt')

GE70F
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
CSH30
IHC
CSR44
Redes De Computadores
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
ED70T
Optativas Isoladas
EEY51
Engenharia Biomedica
CSV30
Processamento Grafico
ES70J
Optativas Isoladas
ED70T
Optativas Isoladas
CSM43
Desenvolvimento Baseado em Plataformas Programacao
FI70D
Fisica
CSM40
Desenvolvimento Baseado em Plataformas Programacao
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSH30


  0%|          | 0/68 [00:00<?, ?it/s]

GE70F
Optativas Isoladas
CSV path: input/dados_estudante/trimmed/90729_trimmed_1.csv
Alg selecionado: Kcliques
DiGraph with 57 nodes and 260 edges
obrigatoria nao cursada: FI71S
obrigatoria nao cursada: CSF20
obrigatoria nao cursada: FI72S
obrigatoria nao cursada: QB70C
obrigatoria nao cursada: EEX11
obrigatoria nao cursada: MA73A
obrigatoria nao cursada: FI73S
obrigatoria nao cursada: EEB21
obrigatoria nao cursada: EEX21
obrigatoria nao cursada: EEF11
obrigatoria nao cursada: EEB22
obrigatoria nao cursada: EEQ31
obrigatoria nao cursada: FI74N
obrigatoria nao cursada: FI74S
obrigatoria nao cursada: CSR31
obrigatoria nao cursada: EEE31
obrigatoria nao cursada: EEF21
obrigatoria nao cursada: ES70G
obrigatoria nao cursada: CSO30
obrigatoria nao cursada: CSW40
obrigatoria nao cursada: CSX54
obrigatoria nao cursada: EEE32
obrigatoria nao cursada: EEX22
obrigatoria nao cursada: EEC21
obrigatoria nao cursada: ES70P
obrigatoria nao cursada: CSS30
obrigatoria nao cursada: EEC31
obrigatoria nao 

  1%|▏         | 1/68 [00:00<00:08,  8.19it/s]

CSH30
IHC
CSV path: input/dados_estudante/trimmed/118844_trimmed_1.csv
Alg selecionado: Kcliques
DiGraph with 55 nodes and 570 edges


...

periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
periodo atual 7
periodo atual 8
periodo atual 9
periodo atual 10
CSB53
Banco De Dados
CSB51
Banco De Dados
CSV30
Processamento Grafico
CSA41
Algoritmos E Complexidade
CSA42
Algoritmos E Complexidade
CSA45
Algoritmos E Complexidade
CSV path: input/dados_estudante/trimmed/190615_trimmed_7.csv
Alg selecionado: Kcliques
DiGraph with 70 nodes and 461 edges
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
periodo atual 7
periodo atual 8
periodo atual 9
periodo atual 10
FI70A
Fisica
CSH30
IHC
CSH42
IHC
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSV path: input/dados_estudante/trimmed/191284_trimmed_7.csv
Alg selecionado: Kcliques
DiGraph with 72 nodes and 494 edges


 97%|█████████▋| 66/68 [00:08<00:00,  7.75it/s]

periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
periodo atual 7
periodo atual 8
periodo atual 9
periodo atual 10
ES70J
Optativas Isoladas
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSM40
Desenvolvimento Baseado em Plataformas Programacao
CSM43
Desenvolvimento Baseado em Plataformas Programacao
CSH30
IHC
CSM41
Desenvolvimento Baseado em Plataformas Programacao
CSH42
IHC
CSV path: input/dados_estudante/trimmed/198372_trimmed_7.csv
Alg selecionado: Kcliques
DiGraph with 74 nodes and 580 edges
obrigatoria nao cursada: CSX43
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
periodo atual 7
periodo atual 8
periodo atual 9
obrigatoria nao cursada - processando: CSX43
periodo atual 10
CSH30
IHC
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSH42
IHC
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSM41
Desenvolvimento Baseado em Plataformas 

100%|██████████| 68/68 [00:08<00:00,  7.89it/s]

periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
periodo atual 7
periodo atual 8
periodo atual 9
periodo atual 10
CSB54
Banco De Dados
EL75H
Engenharia De Software
CSI53
Sistemas Inteligentes
CSI41
Sistemas Inteligentes
CSE40
Engenharia De Software
CSE43
Engenharia De Software
CSV path: input/dados_estudante/trimmed/202261_trimmed_7.csv
Alg selecionado: Kcliques
DiGraph with 72 nodes and 475 edges
obrigatoria nao cursada: CSX54
obrigatoria nao cursada: CSS30
obrigatoria nao cursada: EEC31
obrigatoria nao cursada: EEX23
obrigatoria nao cursada: CSX42
obrigatoria nao cursada: CSX43
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
obrigatoria nao cursada - processando: CSX54
periodo atual 6
periodo atual 7
obrigatoria nao cursada - processando: CSS30
obrigatoria nao cursada - processando: EEX23
periodo atual 8
obrigatoria nao cursada - processando: CSX42
periodo atual 9
obrigatori

In [ ]:
for i in range(1,8):
    teste_recomendacao_grafos(i,'cosseno')
    teste = pd.read_csv('output/res.csv')
    teste.describe().to_csv(f'output/desc_cosseno_{i}.txt')
    teste.drop(teste[teste['RECALL_OP'] == 0].index).describe().to_csv(f'output/desc_notnull_cosseno_{i}.txt')

GE70F
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
CSH30
IHC
CSH30
IHC
CSR44
Redes De Computadores
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
ED70T
Optativas Isoladas
EEY51
Engenharia Biomedica
CSV30
Processamento Grafico
ES70J
Optativas Isoladas
ED70T
Optativas Isoladas
CSM43
Desenvolvimento Baseado em Plataformas Programacao
FI70D
Fisica
CSM40
Desenvolvimento Baseado em Plataformas Programacao
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
ES70N
Optativas Isoladas
ES70N
Optativas Isoladas
CSH30
IHC
MA70C
Otimizacao, Modelos Analiticos e de Simulacao
CSH30


  0%|          | 0/68 [00:00<?, ?it/s]

GE70F
Optativas Isoladas


  1%|▏         | 1/68 [00:01<01:21,  1.22s/it]

obrigatoria nao cursada: FI71S
obrigatoria nao cursada: CSF20
obrigatoria nao cursada: FI72S
obrigatoria nao cursada: QB70C
obrigatoria nao cursada: EEX11
obrigatoria nao cursada: MA73A
obrigatoria nao cursada: FI73S
obrigatoria nao cursada: EEB21
obrigatoria nao cursada: EEX21
obrigatoria nao cursada: EEF11
obrigatoria nao cursada: EEB22
obrigatoria nao cursada: EEQ31
obrigatoria nao cursada: FI74N
obrigatoria nao cursada: FI74S
obrigatoria nao cursada: CSR31
obrigatoria nao cursada: EEE31
obrigatoria nao cursada: EEF21
obrigatoria nao cursada: ES70G
obrigatoria nao cursada: CSO30
obrigatoria nao cursada: CSW40
obrigatoria nao cursada: CSX54
obrigatoria nao cursada: EEE32
obrigatoria nao cursada: EEX22
obrigatoria nao cursada: EEC21
obrigatoria nao cursada: ES70P
obrigatoria nao cursada: CSS30
obrigatoria nao cursada: EEC31
obrigatoria nao cursada: EEX23
obrigatoria nao cursada: CSX42
obrigatoria nao cursada: GE70H
obrigatoria nao cursada: QB70E
obrigatoria nao cursada: CSX43
obrigato

...

obrigatoria nao cursada: CSX43
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
periodo atual 7
periodo atual 8
periodo atual 9
obrigatoria nao cursada - processando: CSX43
periodo atual 10
CSH30
IHC
CSM44
Desenvolvimento Baseado em Plataformas Programacao
CSH42
IHC
CSM45
Desenvolvimento Baseado em Plataformas Programacao
CSM41
Desenvolvimento Baseado em Plataformas Programacao


 99%|█████████▊| 67/68 [01:18<00:01,  1.02s/it]

periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
periodo atual 6
periodo atual 7
periodo atual 8
periodo atual 9
periodo atual 10
CSB54
Banco De Dados
EL75H
Engenharia De Software
CSI53
Sistemas Inteligentes
CSI41
Sistemas Inteligentes
CSE40
Engenharia De Software
CSE43
Engenharia De Software


100%|██████████| 68/68 [01:19<00:00,  1.17s/it]

obrigatoria nao cursada: CSX54
obrigatoria nao cursada: CSS30
obrigatoria nao cursada: EEC31
obrigatoria nao cursada: EEX23
obrigatoria nao cursada: CSX42
obrigatoria nao cursada: CSX43
periodo atual 0
periodo atual 1
periodo atual 2
periodo atual 3
periodo atual 4
periodo atual 5
obrigatoria nao cursada - processando: CSX54
periodo atual 6
periodo atual 7
obrigatoria nao cursada - processando: CSS30
obrigatoria nao cursada - processando: EEX23
periodo atual 8
obrigatoria nao cursada - processando: CSX42
periodo atual 9
obrigatoria nao cursada - processando: CSX43
periodo atual 10
Recall: 

Media: 0.21222222222222223
Minimo: 0
Maximo: 1.0

Precision: 

Media: 0.1906782106782107
Minimo: 0
Maximo: 1.0

Recall Obrigatorias: 

Media: 0.2263888888888889
Minimo: 0
Maximo: 1.0

Precision Obrigatorias: 

Media: 0.31666666666666665
Minimo: 0
Maximo: 1.0

Recall Optativas: 

Media: 0.007936507936507936
Minimo: 0
Maximo: 0.3333333333333333

Precision Optativas: 

Media: 0.016666666666666666
Minim